NAME_1: Marco <br>
LAST-NAME_1: Introvigne <br>
STUDENT-ID_1: 10750466 <br>
NAME_2: Antonio <br>
LAST-NAME_2: Urbano <br>
STUDENT-ID_2: 10527285 <br>
NAME_3: Enrico <br>
LAST-NAME_3: Voltan <br>
STUDENT-ID_3: 10525467 <br>
LEADERBOARD NICKNAME: TLN (Three Little Neurons) <br>


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np 
import pandas as pd
import os
import tensorflow as tf

In [ ]:
SEED = 1234
tf.random.set_seed(SEED) 
np.random.seed(SEED)

cwd = os.getcwd()

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Augmentation


We first tried to train the network without data augmentation. Then we added most meaningfull parameter according to the dataset taking always in consideration the behaviour in order to avoid overfitting.
This final setting is the one which gave us the best result.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rotation_range=10,
                                    zoom_range=0.2,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    brightness_range=[0.1,0.4],
                                    horizontal_flip=True,
                                    fill_mode="nearest",
                                    rescale=1./255)

valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)


In [ ]:
import json
classes = {}
with open('/content/drive/My Drive/Kaggle_1/MaskDataset/train_gt.json') as json_file:
    classes = json.load(json_file)


In [ ]:
dataset_dir = "/content/drive/My Drive/Kaggle_1/MaskDataset/training"
from PIL import Image
df = pd.DataFrame(columns={"name","class"})
classes_names = {
    0:"NOPERSON",
    1:"ALL",
    2:"SOMEONE"
}

for dirname, _, filenames in os.walk(dataset_dir):
    for filename in filenames:
      df2 = pd.DataFrame.from_dict({"name" : [filename], "class" : [classes_names[classes[filename]]]})   
      df = df.append( df2, ignore_index=True)
df

,name,class
0,16649.jpg,NOPERSON
1,16798.jpg,NOPERSON
2,16613.jpg,NOPERSON
3,16507.jpg,NOPERSON
4,16808.jpg,NOPERSON
...,...,...
5609,10868.jpg,SOMEONE
5610,11316.jpg,ALL
5611,11266.jpg,SOMEONE
5612,11186.jpg,ALL


# Training and Validation

We have done crossvalidation splitting the training dataset into training and validation set by keeping the first 5000 images for training and the remaining 614 for validation.
We have done tuning on the image size parameter increasing it gradually.

In [ ]:
img_h = 204
img_w = 204

train_gen = train_data_gen.flow_from_dataframe(
                                                    df[:5000],
                                                    directory=dataset_dir,
                                                    x_col="name",
                                                    y_col="class",
                                                    target_size=(img_h, img_w),
                                                    batch_size=16,
                                                    shuffle=True,
                                                    seed=SEED,
                                                    save_prefix="",
                                                    save_format="png",
                                                    subset=None,
                                                    interpolation="nearest"
                                                ) 

valid_gen = valid_data_gen.flow_from_dataframe(
                                                    df[5000:],
                                                    directory=dataset_dir,
                                                    x_col="name",
                                                    y_col="class",
                                                    target_size=(img_h, img_w),
                                                    batch_size=16,
                                                    shuffle=True,
                                                    seed=SEED,
                                                    save_prefix="",
                                                    save_format="png",
                                                    subset=None,
                                                    interpolation="nearest"
                                                ) 

Found 5000 validated image filenames belonging to 3 classes.
Found 614 validated image filenames belonging to 3 classes.


In [ ]:
num_classes = 3

train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

## Architecture

This is the best network without transfer learning we were able to build after several attempts and it is better described in the report.

In [ ]:
# Architecture

model = tf.keras.Sequential()

# Conv-Conv-Pooling
for i in range(2):
  model.add(tf.keras.layers.Conv2D(filters=128, 
                                   kernel_size=(3,3),
                                   strides=(1,1),
                                   padding='same',
                                   input_shape=[img_h, img_w, 3]))
  model.add(tf.keras.layers.ReLU())
  model.add(tf.keras.layers.Conv2D(filters=128, 
                                  kernel_size=(3,3),
                                  strides=(1,1),
                                  padding='same',
                                  input_shape=[img_h, img_w, 3]))
  model.add(tf.keras.layers.ReLU()
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2),
                                      strides = (2,2)))

#Conv-Pooling
for j in range(2)
  model.add(tf.keras.layers.Conv2D(filters=128, 
                                  kernel_size=(3,3),
                                  strides=(1,1),
                                  padding='same',
                                  input_shape=[img_h, img_w, 3]))
  model.add(tf.keras.layers.ReLU())
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2),
                                      strides = (2,2)))

# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Visualize 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 204, 204, 128)     3584      
_________________________________________________________________
re_lu_6 (ReLU)               (None, 204, 204, 128)     0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 204, 204, 128)     147584    
_________________________________________________________________
re_lu_7 (ReLU)               (None, 204, 204, 128)     0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 102, 102, 128)     0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 102, 102, 128)     147584    
_________________________________________________________________
re_lu_8 (ReLU)               (None, 102, 102, 128)    

## Training Model

We tried several optimizer but the best one were Adam and SGD. We did tuning on the learning rate for both of optimizers, and the best result was with Adam with learning rate equal to 0.0001.

In [ ]:
lr = 1e-4
model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

We tried different solutions to prevent overfitting (Early stopping, Dropout).
We adopt the early stopping technique and changed the patiance several times.

In [ ]:
# Early Stopping
callbacks = []
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

In [ ]:
model.fit(x=train_dataset,
          epochs=100,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen),
          callbacks=callbacks)

Epoch 1/100
200/200 [==============================] - ETA: 0s - loss: 1.1009 - accuracy: 0.3458WARNING:tensorflow:From <ipython-input-19-2492aac2d56d>:40: Model.predict_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.predict, which supports generators.
200/200 [==============================] - 144s 722ms/step - loss: 1.1009 - accuracy: 0.3458 - val_loss: 1.0992 - val_accuracy: 0.2818
Epoch 2/100
200/200 [==============================] - 143s 715ms/step - loss: 1.0916 - accuracy: 0.3724 - val_loss: 1.0833 - val_accuracy: 0.3795
Epoch 3/100
200/200 [==============================] - 143s 716ms/step - loss: 1.0723 - accuracy: 0.4182 - val_loss: 1.0277 - val_accuracy: 0.4544
Epoch 4/100
200/200 [==============================] - 140s 701ms/step - loss: 1.0264 - accuracy: 0.4678 - val_loss: 1.2273 - val_accuracy: 0.4902
Epoch 5/100
200/200 [==============================] - 138s 69

## Test



In [ ]:
test_dir = "/content/drive/My Drive/Kaggle_1/MaskDataset/test_images/"

In [ ]:
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             batch_size=8, 
                                             target_size=(img_h, img_w),
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], None))

Found 450 images belonging to 1 classes.


In [ ]:
eval_out = model.predict(x=test_dataset,
                          steps=len(test_gen),
                          verbose=0)
eval_out

array([[7.6152164e-01, 4.9786750e-06, 2.3847336e-01],
       [2.6056604e-03, 4.2759824e-01, 5.6979603e-01],
       [9.8858917e-01, 3.7545960e-06, 1.1407030e-02],
       ...,
       [1.6364202e-01, 1.1777764e-03, 8.3518016e-01],
       [3.0051518e-05, 9.9900430e-01, 9.6570019e-04],
       [2.8314272e-01, 2.8652452e-02, 6.8820482e-01]], dtype=float32)

In [ ]:
label_map = (train_gen.class_indices)

map_cat = {v: k for k, v in label_map.items()}

real_names = {v: k for k, v in classes_names.items()}

results = {}
for i,name in enumerate(test_gen.filenames):
  real_name = name.split("/")[1]
  category = map_cat[np.argmax(eval_out[i])]
  results[real_name] = real_names[category]

results

{'10001.jpg': 1,
 '10040.jpg': 2,
 '10074.jpg': 1,
 '10084.jpg': 1,
 '10100.jpg': 2,
 '10120.jpg': 2,
 '10125.jpg': 1,
 '10148.jpg': 1,
 '10213.jpg': 1,
 '10239.jpg': 1,
 '10242.jpg': 1,
 '10259.jpg': 1,
 '10289.jpg': 2,
 '10296.jpg': 0,
 '10323.jpg': 2,
 '10324.jpg': 0,
 '10326.jpg': 2,
 '10328.jpg': 0,
 '10330.jpg': 0,
 '10346.jpg': 2,
 '10349.jpg': 1,
 '10370.jpg': 1,
 '10382.jpg': 1,
 '10396.jpg': 1,
 '10411.jpg': 1,
 '10437.jpg': 2,
 '10459.jpg': 0,
 '10473.jpg': 1,
 '10477.jpg': 1,
 '10494.jpg': 0,
 '10520.jpg': 2,
 '10530.jpg': 2,
 '10571.jpg': 1,
 '10572.jpg': 1,
 '10581.jpg': 2,
 '10620.jpg': 0,
 '10641.jpg': 2,
 '10643.jpg': 1,
 '10662.jpg': 0,
 '10691.jpg': 1,
 '10715.jpg': 2,
 '10727.jpg': 1,
 '10760.jpg': 0,
 '10771.jpg': 1,
 '10782.jpg': 2,
 '10800.jpg': 0,
 '10812.jpg': 1,
 '10845.jpg': 2,
 '10850.jpg': 2,
 '10853.jpg': 1,
 '10863.jpg': 0,
 '10873.jpg': 0,
 '10999.jpg': 1,
 '11012.jpg': 1,
 '11026.jpg': 0,
 '11045.jpg': 1,
 '11115.jpg': 0,
 '11130.jpg': 2,
 '11136.jpg': 

## Kaggle Result

In [ ]:
#Kaggle Result

import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
create_csv(results,"/content/drive/My Drive/Keras3")

FINISH!
